#  LaTeX-OCR colab reproduction with pytorch lightning

14-09-2023

using new trainer file, modified config file and modified dataset file

manually upload/put these 3 files under /content

In [1]:
!pwd

/content


In [2]:
%load_ext autoreload
%autoreload 2

# installs

In [3]:
from google.colab import files
import os
from os import path as osp

In [4]:
# for laziness
!pip install pix2tex[train] -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.3 MB/s eta 0:00:00


In [5]:
!pip install --upgrade --no-cache-dir gdown -q
!pip install timm==0.5.4 -U -q
!pip install opencv-python-headless -U -q
!pip install lightning==2.0.8 -q
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.0/660.0 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# clone repo

In [6]:
%cd /content
!git clone https://github.com/lukas-blecher/LaTeX-OCR

/content
Cloning into 'LaTeX-OCR'...
remote: Enumerating objects: 1831, done.
remote: Counting objects: 100% (714/714), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 1831 (delta 568), reused 473 (delta 473), pack-reused 1117
Receiving objects: 100% (1831/1831), 9.40 MiB | 17.45 MiB/s, done.
Resolving deltas: 100% (1042/1042), done.


# form dataset

brief notes on dataset format

- text file contains space-separated latex symbols, each line corresponds to a figure

- image files must be named according to line number, line 1 -> 00000.png, line 2 -> 00001.png, etc.

- image files can correspond to a subset of math lines, as below

In [7]:
%%time
if not osp.exists("/content/pdf"):
    !mkdir /content/pdf
    %cd /content/pdf
    !gdown -O pdf.zip 176PKaCUDWmTJdQwc-OfkO0y8t4gLsIvQ
    !gdown -O math.txt 1QUjX6PFWPa-HBWdcY-7bA5TRVUnbyS1D
    !unzip -q pdf.zip
    %cd /content

/content/pdf
Downloading...
From (uriginal): https://drive.google.com/uc?id=176PKaCUDWmTJdQwc-OfkO0y8t4gLsIvQ
From (redirected): https://drive.google.com/uc?id=176PKaCUDWmTJdQwc-OfkO0y8t4gLsIvQ&confirm=t&uuid=320b25e5-22bd-40bc-a58e-da3897c9f03c
To: /content/pdf/pdf.zip
100% 284M/284M [00:08<00:00, 34.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QUjX6PFWPa-HBWdcY-7bA5TRVUnbyS1D
To: /content/pdf/math.txt
100% 36.6M/36.6M [00:01<00:00, 33.3MB/s]
/content
CPU times: user 195 ms, sys: 35.7 ms, total: 230 ms
Wall time: 28.3 s


In [8]:
%cd /content/LaTeX-OCR
!python -m pix2tex.dataset.dataset -i /content/pdf/train -e /content/pdf/math.txt \
    -o /content/pdf/train.pkl

/content/LaTeX-OCR
Generate dataset
100% 158480/158480 [00:02<00:00, 56494.95it/s]


In [9]:
%cd /content/LaTeX-OCR
!python -m pix2tex.dataset.dataset -i /content/pdf/val -e /content/pdf/math.txt \
    -o /content/pdf/val.pkl

/content/LaTeX-OCR
Generate dataset
100% 6765/6765 [00:00<00:00, 31910.39it/s]


In [10]:
%cd /content/LaTeX-OCR
!python -m pix2tex.dataset.dataset -i /content/pdf/test -e /content/pdf/math.txt \
    -o /content/pdf/test.pkl

/content/LaTeX-OCR
Generate dataset
100% 30637/30637 [00:00<00:00, 54932.35it/s]


# load whole log directory from drive (for continue training only)

In [11]:
from google.colab import drive
drive.mount('/content/drive')

import shutil
from os import path as osp

drive_dir = "/content/drive/MyDrive/latex_ocr" # the partent dir in drive, change it
log_dir = "reproducing/version_1"

src = osp.join(drive_dir, log_dir)
dst = osp.join("/content/lightning_logs", log_dir)
shutil.copytree(src, dst)

drive.flush_and_unmount()

Mounted at /content/drive


# run

1. edit hyper parameters and/or I/O locations!

2. move the new trainer and modified dataset file

3. run

pt-lightning:

- would assign a version number if not supplied, the string would be "version_0", etc.

- requires a correct dataset size to trigger validation loop (only triggered after "all" training pairs are iterated)

In [12]:
# open the file in a new tab in colab
# if contine training, change the load_pl_chpt, version number, middlestop accordingingly
from google.colab import files
files.view("/content/colab_config.yaml")

<IPython.core.display.Javascript object>

In [14]:
import shutil
shutil.copy2("/content/colab_train_pl.py", "/content/LaTeX-OCR/pix2tex/train_pl.py")
# copy original dataset script, for reference only
if not osp.exists("/content/dataset_original.py"):
    shutil.copy2("/content/LaTeX-OCR/pix2tex/dataset/dataset.py", "/content/dataset_original.py")
shutil.copy2("/content/pl_dataset.py", "/content/LaTeX-OCR/pix2tex/dataset/dataset.py")
%cd /content/LaTeX-OCR
!python -m pix2tex.train_pl --config /content/colab_config.yaml

# store to drive, copied from following section
from google.colab import drive
drive.mount('/content/drive')

import shutil
src = "/content/lightning_logs/reproducing/version_1"
dst = "/content/drive/MyDrive/latex_ocr/reproducing/version_1"
shutil.copytree(src, dst, dirs_exist_ok=True)

drive.flush_and_unmount()

/content/LaTeX-OCR
2023-09-14 11:26:43.195857: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
iterating through dataset to get actual size:  99% 4891/4941 [03:59<00:02, 20.45it/s]
iterating through dataset to get actual size:  99% 468/471 [00:05<00:00, 90.77it/s] 
Global seed set to 42
out path = /content/lightning_logs
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /content/lightning_logs/reproducing/version_1/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at /content/lightning_logs/reproducing/version_1/checkpoints/epoch=1-step=9782.ckpt
LOCAL_RANK: 0 - CUDA

# store to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import shutil
src = "/content/lightning_logs/reproducing/version_1"
dst = "/content/drive/MyDrive/latex_ocr/reproducing/version_1"
shutil.copytree(src, dst, dirs_exist_ok=True)

drive.flush_and_unmount()

# turn ckpt into pth

In [ ]:
import torch
from tqdm import tqdm
from os import path as osp

def ckpt2pth(cpath, pthpath):
    mdict = torch.load(cpath)["state_dict"]
    nm_dict = dict()
    for k in tqdm(mdict.keys()):
        assert k.find("pt_model.") != -1
        nk = k.split("pt_model.")[1]
        nm_dict[nk] = mdict[k]
    torch.save(nm_dict, pthpath)
    print(f"successfully saved at {osp.realpath(pthpath)}")


In [ ]:
# change them
# cpath = "/content/lightning_logs/reproducing/version_1/checkpoints/ep1.pth"
# pthdesc = "/content/ep1.pth"

# ckpt2pth(cpath=cpath, pthpath=pthdesc)

# evaluate by given script

In [ ]:
# time varies from batch to batch and model to model a lot
%cd /content/LaTeX-OCR
!python /content/LaTeX-OCR/pix2tex/eval.py \
    -c /content/lightning_logs/reproducing/version_1/checkpoints/ep1.pth \
    -d /content/pdf/val.pkl \
    -b 16 \
    -n 20

# tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs